#### Introduction

The purpose of this notebook, along with 01_data_setup_example.ipynb and 03_demo_explore_results.ipynb, is to provide a tutorial of how you may want to use the popexposure pacakge functions.

Please see 01_data_setup_example.ipynb before you work through this notebook!

#### Outline

To recap, our goal was to demo all the options available in popexposure. In this notebook we'll do five separate things, which align with the five options available in the package. 

First we'll do:

0. Setup, and then:

1. Find the total number of people residing within 10km of one or more California wildfire 
disasters in 2016, 2017, and 2018. 
2. Find the total number of people residing within 10 km of each unique California wildfire
disaster in 2016, 2017, and 2018.
3. Find the total number of people residing within 10km of one or more California wildfire 
disaster in 2016, 2017, and 2018 by 2020 ZCTA. 
4. Find the total number of people residing within 10 km of each unique California wildfire
disaster in 2016, 2017, and 2018 by 2020 ZCTA.
5. Find the population of all 2020 ZCTAs. 

In the last notebook, we prepared the wildfire disaster exposure data and ZCTA data to pass to the popexposure functions so we could complete these computations. Here, we'll complete each of them in this order. 

#### 0. Setup

We need to import some libraries and also install and import popexposure. If you haven't installed popexposure in the environment you're working in now, go ahead and activate that environment, and pip install popexposure in the terminal. popexposure is included in the pop_exp environment for this tutorial. We can then import the functions within popexposure. 

In [ ]:
# We start by importing necessary libraries.
import pathlib
import sys
import glob
import pandas as pd
# Here's the popexposure import 
from popexposure import find_exposure as ex

We'll also set some paths to make it easy to access the data we cleaned for 
this tutorial. 

To find the number of people affected by one or more CA wildfire disaster by year 2016-2018, and by ZCTA, we need to get the paths to each of our wildfire files that we made in the data setup notebook.

The regular expression below selects all the files in the interim data directory that have 'fire' in the name. 

In [ ]:
# Paths 
base_path = pathlib.Path.cwd().parent
data_dir = base_path / "demo_data"
# wf paths regex
wildfire_paths = glob.glob(str(data_dir / "02_interim_data" / "*fire*"))

We also need the path to the population raster we're using, and the ZCTA file.

In [ ]:
# GHSL pop raster
ghsl_path = data_dir / "01_raw_data" / "GHS_POP_E2020_GLOBE_R2023A_54009_100_V1_0_R5_C8.tif"

# ZCTA path 
zcta_path = glob.glob(str(data_dir / "02_interim_data" / "*zcta*"))[0]

We're now set up to run the five cases we're interested in. 

#### 1. Find the total number of people residing within 10km of one or more California wildfire disasters in 2016, 2017, and 2018. 

Our first goal was to find the total number of people residing within 10 km of one or more California wildfire disaster in 2016, 2017, and 2018.

To do this, we can run prep_data and then exposed_pop with the parameter hazard_specific = False.

Because we're looping over three years, we'll initialize an empty list first, and then store the results in this list. We're also adding a year variable to the result as we go. In total, this takes around 5 seconds.


In [ ]:
est = ex.PopEstimator()

num_exposed_list = []
start_year = 2016

for i in range(0, 3):
    year_dat = est.prep_data(path_to_data=wildfire_paths[i], geo_type='hazard')
    num_exposed = est.exposed_pop(pop_path=ghsl_path, 
                                  hazards = year_dat,
                                  hazard_specific=False)
    num_exposed['year'] = start_year + i
    num_exposed_list.append(num_exposed)

Because we added a year variable to each output, we can concatonate these dataframes together, and then look at the output. 

In [ ]:
# Join those dataframes together. 
num_exposed_df = pd.concat(num_exposed_list, axis=0)
num_exposed_df.head()

Our output has three columns: ID_hazard, exposed_10, and year. 
We added year, but the other two are output from exposed_pop.
exposed_10 is called that because we named our buffer_dist column buffer_dist_10, so that suffix got carried through to our results.
Because we ran the function with hazard_specific = False, our ID_hazard column has changed. It now says 'merged_geoms', which means that we got one number representing the count of everyone exposed to one or more wildifre disasters in each year, so the IDs are no longer wildfire IDs.

We wanted to count the number of people residing within 10km of *any* California wildfire disaster. There are some people who live within 10km of two or more wildfire disasters. Because we just wanted the total, we did not want to  double count those people. When computing a total, rather than the number of people affected by each unique hazard, exposed_pop takes the unary union of any buffered hazards that are overlapping, and finds the total of 
everyone residing within that area.

We can save the output.

In [ ]:
num_exposed_df.to_parquet(data_dir / "03_results" / "num_people_affected_by_wildfire.parquet")

#### 2. Find the total number of people residing within 10 km of each unique California wildfire disaster in 2016, 2017, and 2018.

We also wanted to 2. Find the total number of people residing within 10 km of each unique California wildfire disaster in 2016, 2017, and 2018. 

To do this, we also need to use exposed_pop, with all the same arguments except for hazard_specific. In this case, we set hazard_specific to True. This means that we will count the number of people within 10km of each wildfire disaster boundary, regardless of whether two or more exposed areas overlap. 

In [ ]:
est = ex.PopEstimator()

num_exposed_list = []
start_year = 2016

for i in range(0, 3):
    year_dat = est.prep_data(path_to_data=wildfire_paths[i], geo_type='hazard')
    num_exposed = est.exposed_pop(pop_path=ghsl_path, 
                                  hazards = year_dat,
                                  hazard_specific=True)
    num_exposed['year'] = start_year + i
    num_exposed_list.append(num_exposed)

num_exposed_unique = pd.concat(num_exposed_list, axis=0)
num_exposed_unique.head()

Again, our output has three columns: ID_hazard, exposed_10, and year. 

This time, the ID_hazard column is the same as the one we passed to this 
function. This time, if people lived within 10 km of one or more fires, they
are counted in the total people affected by that fire. This means that exposed_pop
returns a dataframe with one row per hazard ID, and people may
be double counted or triple or more. 

We can save the output.

In [ ]:
num_exposed_unique.to_parquet(data_dir / "03_results" / "num_aff_by_unique_wildfire.parquet")

There were two more ways we wanted to define exposure. 

3. Find the total number of people residing within 10km of one or more California wildfire 
disasters in 2016, 2017, and 2018 by 2020 ZCTA. 
4. Find the total number of people residing within 10 km of each unique California wildfire
disaster in 2016, 2017, and 2018 by 2020 ZCTA.

These are analogous to the two quantities we just computed, but this time, we want to know these exposures by ZCTA. 

To do this, we need to run exposed_pop again but this time with additional administrative geographies: ZCTAs.


#### 3. Find the total number of people residing within 10km of one or more California wildfire disaster in 2016, 2017, and 2018 by 2020 ZCTA. 
To do "3. Find the total number of people residing within 10km of one or more California wildfire disaster in 2016, 2017, and 2018 by 2020 ZCTA", we'll run run exposed_pop with hazard_specific = False, and we'll set the optional argument 'spatial_units' to a dataframe of 2020 ZCTAs.

We need to prepare the ZCTA data once first before we include it! We'll run the prep_data function once to prepare the ZCTA data, and then use that prepared data in every interation of the loop over years 2016-2018.


In [ ]:
num_exposed_zcta_list = []
start_year = 2016

# prepare zcta data
zctas = est.prep_data(path_to_data=zcta_path, geo_type='spatial_unit')

for i in range(0, 3):
    hazards = est.prep_data(path_to_data=wildfire_paths[i], geo_type='hazard')
    num_exposed_zcta = est.exposed_pop(pop_path=ghsl_path, hazard_specific=False, hazards=hazards, spatial_units=zctas)
    num_exposed_zcta['year'] = start_year + i
    num_exposed_zcta_list.append(num_exposed_zcta)

This computation took about 8 seconds - a little bit longer than when we weren't looking for ZCTA-specific estimates. 

In [ ]:
# putting all years into one dataframe
num_affected_zcta_df = pd.concat(num_exposed_zcta_list, axis=0)
num_affected_zcta_df.head()

In [ ]:
# And we can save 
num_affected_zcta_df.to_parquet(data_dir / "03_results" / "num_people_affected_by_wildfire_by_zcta.parquet")
num_affected_zcta_df.head()

#### 4. Find the total number of people residing within 10 km of each unique California wildfire disaster in 2016, 2017, and 2018 by 2020 ZCTA.
For our final case of counting exposed people, we wanted to find the number of people living near each unique hazard by each ZCTA. For this, we need to use exposed_pop in the same way that we just did, but with hazard_specific = True.  We already prepared the zcta data with prep_data in the previous step, so we can just use that data again. 

In [ ]:
num_exposed_zcta_unique_list = []
start_year = 2016

for i in range(0, 3):
    hazards = est.prep_data(path_to_data=wildfire_paths[i], geo_type='hazard')
    num_exposed_zcta_unique = est.exposed_pop(pop_path=ghsl_path, 
                                               hazard_specific=True,
                                               hazards=hazards, 
                                               spatial_units=zctas # using zctas from previous prep_data call.

    )
    
    num_exposed_zcta_unique['year'] = start_year + i
    num_exposed_zcta_unique_list.append(num_exposed_zcta_unique)

# all years in one dataframe
num_exposed_df_zcta_unique = pd.concat(num_exposed_zcta_unique_list, axis=0)

# and we can save
num_exposed_df_zcta_unique.to_parquet(data_dir / "03_results" / "num_people_affected_by_wildfire_zcta_unique.parquet")

In [ ]:
num_exposed_df_zcta_unique

We will explore some of the output from these runs in the next section of the tutorial.  

#### 5. Find the population of all 2020 ZCTAs. 

Finally, let's use the function pop to get some denominators for our dataset. This function can help us use the gridded population data we used to find the number of people residing within the hazard buffers to also find the number of people residing in each ZCTA. This is useful if we're using a gridded population dataset that we think is a big improvement over other population counts in our additional spatial units, or we just want to be consistent. 

To call this function, all we need to do is use the same paths we've used previously:

In [ ]:
num_residing_by_zcta = est.pop(pop_path=ghsl_path, spatial_units=zctas)
num_residing_by_zcta.to_parquet(data_dir / "03_results" / "num_people_residing_by_zcta.parquet")
num_residing_by_zcta.head()



This time, we have a column for spatial unit and a column for the number of people living in that spatial unit. 
Please continue to part 3 of this tutorial to explore the output of these functions! 
